In [1]:
import pandas as pd
import numpy as np

/var/folders/y3/8t9j98gs2hx7slww07_ldb1w0000gn/T/ipykernel_15638/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Import the CSV file users.csv and assign it to the variable users
users = pd.read_csv('users.csv')
users.head()

,user_id,timestamp,device_id,clicked
0,U790620,2022-01-01 00:23:33 Saturday,M003,False
1,U584867,2022-01-01 01:30:07 Saturday,M001,False
2,U128681,2022-01-01 01:30:14 Saturday,M002,True
3,U694898,2022-01-01 01:31:55 Saturday,M003,False
4,U456823,2022-01-01 03:18:25 Saturday,M001,False


In [3]:
#Import the CSV file advertisements.csv and assign it to the variable advertisements
advertisements = pd.read_csv('advertisements.csv')
advertisements.head()

,user_id,timestamp,ad_source,ad_version
0,U790620,2022-01-01 00:23:33 Saturday,Twitter,A
1,U584867,2022-01-01 01:30:07 Saturday,Google,B
2,U128681,2022-01-01 01:30:14 Saturday,TikTok,B
3,U694898,2022-01-01 01:31:55 Saturday,TikTok,A
4,U456823,2022-01-01 03:18:25 Saturday,Google,A


In [4]:
# There are two user_id columns, one in each DataFrame. Let's do a quick check to see if they have the same number of unique users. If they don't, we'll know to be careful about this issue later on in our analysis.
# Count the unique number of user_ids in users and separately in advertisements
user_count = users['user_id'].nunique()
ad_users = advertisements['user_id'].nunique()

print(user_count)
print(ad_users)

15122
14602


In [5]:
#Merge users.csv and advertisements.csv in order to determine if a user "clicked" and which ad they viewed
users_ads = pd.merge(left = users, right = advertisements, left_on = ['user_id', 'timestamp'] , right_on = ['user_id', 'timestamp'], how = 'inner' )
users_ads.head()

,user_id,timestamp,device_id,clicked,ad_source,ad_version
0,U790620,2022-01-01 00:23:33 Saturday,M003,False,Twitter,A
1,U584867,2022-01-01 01:30:07 Saturday,M001,False,Google,B
2,U128681,2022-01-01 01:30:14 Saturday,M002,True,TikTok,B
3,U694898,2022-01-01 01:31:55 Saturday,M003,False,TikTok,A
4,U456823,2022-01-01 03:18:25 Saturday,M001,False,Google,A


In [6]:
#To begin our exploration, let's count how many times each ad_version was viewed by users in users_ads.
ad_view_count = users_ads.groupby('ad_version').agg({'user_id':'count'})
ad_view_count.columns = ['num_views']
ad_view_count.reset_index()

,ad_version,num_views
0,A,7154
1,B,7270


In [7]:
#Add another row to account for users who saw an add more than once 
ad_view_count = users_ads.groupby('ad_version').agg({'user_id':['count','nunique']})
ad_view_count.columns = ['num_views', 'nunique']
ad_view_count.reset_index()

,ad_version,num_views,nunique
0,A,7154,7125
1,B,7270,7232


In [8]:
#Calculate the click through rate for the percentage of users who clicked on each add
ad_ctr_pct = users_ads.groupby('ad_version').agg({'clicked':'mean'})*100
ad_ctr_pct.columns = ['click_rate']
ad_ctr_pct.reset_index()

,ad_version,click_rate
0,A,12.412636
1,B,19.422283


Compare Ad Performance by Social Media Platform:

In [9]:
#Click through rate by social media platform 
social_platform = users_ads.groupby(['ad_source', 'ad_version']).agg({'clicked':'mean'})*100
social_platform.columns = ['ctr']
social_platform.reset_index()

,ad_source,ad_version,ctr
0,Google,A,12.838469
1,Google,B,19.882512
2,Meta,A,12.966418
3,Meta,B,18.803419
4,TikTok,A,11.581921
5,TikTok,B,20.235849
6,Twitter,A,11.962366
7,Twitter,B,17.507003


In [11]:
#Convert to wide format using a pivot table 
ad_social = pd.pivot_table(users_ads,
    index = 'ad_source',
    columns = 'ad_version',
    values = 'clicked',
    aggfunc = 'mean'
    )
ad_social

ad_version,A,B
ad_source,,
Google,0.128385,0.198825
Meta,0.129664,0.188034
TikTok,0.115819,0.202358
Twitter,0.119624,0.175070


Compare Ad Performance by Tech Device:

In [12]:
#Add another file containing device information 
devices = pd.read_csv('devices.csv')
devices

,device_id,device_type,brand
0,M001,Mobile,Apple
1,M002,Mobile,Samsung
2,M003,Mobile,Google
3,M004,Mobile,Huawei
4,M005,Mobile,Xiaomi
5,M006,Mobile,vivo
6,P001,PC,Apple
7,P002,PC,Dell
8,P003,PC,HP
9,P004,PC,ASUS


In [13]:
#Merge table to compare ad information to device information 
users_devices = pd.merge(left = users_ads, right = devices, left_on = 'device_id' , right_on = 'device_id' , how = 'inner' )
users_devices.head()

,user_id,timestamp,device_id,clicked,ad_source,ad_version,device_type,brand
0,U790620,2022-01-01 00:23:33 Saturday,M003,False,Twitter,A,Mobile,Google
1,U584867,2022-01-01 01:30:07 Saturday,M001,False,Google,B,Mobile,Apple
2,U128681,2022-01-01 01:30:14 Saturday,M002,True,TikTok,B,Mobile,Samsung
3,U694898,2022-01-01 01:31:55 Saturday,M003,False,TikTok,A,Mobile,Google
4,U456823,2022-01-01 03:18:25 Saturday,M001,False,Google,A,Mobile,Apple


In [14]:
#Calculate the percentage of users who clicked on an ad based on their device type and ad version 
user_info = pd.pivot_table(users_devices, index = 'device_type', columns = 'ad_version', values = 'clicked', aggfunc = 'mean')
user_info

ad_version,A,B
device_type,,
Mobile,0.121057,0.215877
PC,0.121311,0.182425
Tablet,0.128315,0.171348


Weekday and Weekend Performance by Device Type

In [15]:
#Let's break our analysis down further by weekday versus weekend user behavior using the SAC method 


# Create 'day_of_week' column
users_devices['day_of_week'] = users_devices['timestamp'].str.split(' ', expand=True)[2]


#Count the number of users who viewed each ad, grouped by the day of the week and the ad version they saw

# weekend_count = users_devices.groupby(['day_of_week', 'ad_version']).agg({'clicked':'count'})
# weekend_count.head(20)

#Convert to pivot table
weekend_count_p = pd.pivot_table(users_devices, index = 'day_of_week', columns = 'ad_version', values = 'clicked', aggfunc = 'count')
weekend_count_p

ad_version,A,B
day_of_week,,
Friday,1009,999
Monday,953,977
Saturday,983,934
Sunday,926,987
Thursday,950,932
Tuesday,937,994
Wednesday,986,989


In [16]:
#Split user_devices into two dataframes (weekend and weekdays)

is_weekend = (users_devices['day_of_week'] == 'Saturday') | (users_devices['day_of_week'] == 'Sunday')
weekdays = (users_devices[~ is_weekend])
weekend = users_devices[is_weekend]
weekend

,user_id,timestamp,device_id,clicked,ad_source,ad_version,device_type,brand,day_of_week
0,U790620,2022-01-01 00:23:33 Saturday,M003,False,Twitter,A,Mobile,Google,Saturday
1,U584867,2022-01-01 01:30:07 Saturday,M001,False,Google,B,Mobile,Apple,Saturday
2,U128681,2022-01-01 01:30:14 Saturday,M002,True,TikTok,B,Mobile,Samsung,Saturday
3,U694898,2022-01-01 01:31:55 Saturday,M003,False,TikTok,A,Mobile,Google,Saturday
4,U456823,2022-01-01 03:18:25 Saturday,M001,False,Google,A,Mobile,Apple,Saturday
...,...,...,...,...,...,...,...,...,...
13551,U440244,2022-12-31 22:35:32 Saturday,M001,False,Twitter,A,Mobile,Apple,Saturday
13552,U816412,2022-12-31 22:50:53 Saturday,P005,False,Google,A,PC,Lenovo,Saturday
13553,U598035,2022-12-31 23:39:59 Saturday,T001,False,Google,A,Tablet,Apple,Saturday
13554,U746551,2022-12-31 23:46:15 Saturday,P006,False,Google,A,PC,Other,Saturday


In [17]:
#Weekday click through rate and amount of views
weekday_ctr = weekdays.groupby(['device_type', 'ad_version']).agg({'clicked':['count', 'mean']})
weekday_ctr.columns = ['weekday_views', 'weekday_rate']
weekday_ctr.reset_index()


,device_type,ad_version,weekday_views,weekday_rate
0,Mobile,A,2517,0.125944
1,Mobile,B,2594,0.212799
2,PC,A,1309,0.128342
3,PC,B,1285,0.189105
4,Tablet,A,1009,0.142716
5,Tablet,B,1012,0.164032


In [18]:
#Weekend click through rate and amount of views 
weekend_ctr = weekend.groupby(['device_type', 'ad_version']).agg({'clicked':['count', 'mean']})
weekend_ctr.columns = ['weekend_views', 'weekend_rate']
weekend_ctr.reset_index()

,device_type,ad_version,weekend_views,weekend_rate
0,Mobile,A,1002,0.108782
1,Mobile,B,996,0.223896
2,PC,A,521,0.103647
3,PC,B,513,0.165692
4,Tablet,A,386,0.090674
5,Tablet,B,412,0.189320


In [19]:
#Combine the weekend and weekday dataframes for comparison 
combined_ctr = pd.merge(left = weekday_ctr, right = weekend_ctr, left_on = ['device_type','ad_version'], right_on = ['device_type','ad_version'], how = 'inner')
combined_ctr.head()

weekday_views  weekday_rate  weekend_views  \
device_type ad_version                                               
Mobile      A                    2517      0.125944           1002   
            B                    2594      0.212799            996   
PC          A                    1309      0.128342            521   
            B                    1285      0.189105            513   
Tablet      A                    1009      0.142716            386   

                        weekend_rate  
device_type ad_version                
Mobile      A               0.108782  
            B               0.223896  
PC          A               0.103647  
            B               0.165692  
Tablet      A               0.090674